In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
import time
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np

# Academic Ranking of World Universities
# Shanghai Ranking

## Web scraping dos dados do ranking Shanghai

Para o web scraing dos dados será usada a biblioteca selenium

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [35]:
ano='2005'
website='https://www.shanghairanking.com/rankings/arwu/'+ano
driver = webdriver.Chrome()
driver.get(website) #abre a pagina
driver.maximize_window()  #maximizar a página

Na primeira coluna da tabela estão os seguintes headers:

- World Rank;
- Institution;
- Country/Region;
- National/Regional Rank;
- Total Score;
- Menu com as feutures:
    - Alumni;
    - Award;
    - HiCi;
    - N&S;
    - PUB;
    - PCP.

As features estão em um menu dropdown. Portanto durante o processo de scraping deverá ocorrer interação com esse menu para que ser possível coletar os dados para cada uma delas. Além disso, os dados para todas as universidades estão 
em diversas páginas, sendo assim será necessário interagir com a numeração das páginas para coletar os dados do completos do ranking


In [36]:
features=["Alumni",'Award','HiCi','N&S','PUB','PCP'] #features calculadas cada universidade. O significado de cada uma dela pode ser encontrado no notebook com a análise dos dados dos rankings

#Dicionário que receberá os dados
ranking={'World Rank' :[],
              'Institution':[],
              'Country'    :[],
              'Regional Rank':[],
              'Total Score':[],
                'Alumni': [],
                'Award' : [],
                'HiCi'  : [],
                'N&S'  : [],
                'PUB'   : [],
                'PCP'   : []}

path_to_save='/home/rodolfo/Insync/rodolfopcruz2@gmail.com/Google Drive/Estudo/Python_Projects/education_data/dados/Shanghai World University Rankings/WEB_SCRAPPING_SHANGHAI/'
 

In [37]:
#número de páginas
pagination=driver.find_element('xpath',"//ul[contains(@class,'pagination')]")
paginas=pagination.find_elements('xpath','./li')
num_paginas=paginas[-2].text
num_paginas

'17'

In [38]:
botao_next=pagination.find_element('xpath','//li[@class=" ant-pagination-next"]')
time.sleep(3)

pagina_atual=1
while pagina_atual<=int(num_paginas):

    botoes_header=driver.find_elements(by='xpath',value='//th//img') #localizar botão dropdown. Ele está nos headers da tabela. Ao clicar nesse botão o menu dropdow é aberto
    botoes_header=botoes_header[-1] #nos headers estão dois botões dropdown. O primeiro seleciona o país e o segundo a feature. O botão desejado é o segundo.
        
    opcoes=driver.find_elements(by='xpath',value='//th') #menus dropdown no header da tabela
    opcoes=opcoes[-1] #segundo menu dropdown no header da tabela
    opcoes=opcoes.find_elements('xpath','.//li') #pegar as opções contidas no menu dropdown. Opcoes é uma lista em que cada elemento corresponde a uma feature
        
    table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem os header
    table_rows=table.find_elements(by='xpath',value="./tr") #rows da tabela

    first_feature=True
    for feature in features:       
        botoes_header.click() #clicar no botão para abrir o menu dropdown
        time.sleep(1)
        for opcao in opcoes: #cada opcao é uma das features no menu dropdown
            if opcao.text==feature: #opcao.text retorna o nome da feature. Seleciona a opção desejada da feature
                opcao.click()
                if first_feature:
                    for row in table_rows: #loop por todas as linhas da tabela
                        td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                        ranking['World Rank'].append(td[0].text)#primeira coluna é a posicao no ranking mundial
                        ranking['Institution'].append(td[1].text)#primeira coluna é a posicao no ranking mundial
                        codigo_pais=td[2].find_element(by='xpath',value='div[@class="region-img"]')
                        ranking['Country'].append(codigo_pais.get_attribute("style").split('/')[-1].split('.')[0])
                        ranking['Regional Rank'].append(td[3].text)
                        ranking['Total Score'].append(td[4].text)
                        ranking[feature].append(td[5].text)
                        first_feature=False
                else:
                    for row in table_rows: #loop por todas as linhas da tabela
                        td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                        ranking[feature].append(td[5].text)
                
                time.sleep(5)
                break
    time.sleep(3)  
    botao_next.click()
    time.sleep(3)
    botao_top=driver.find_element('xpath','//div[@class="back-top"]')
    botao_top.click()
    time.sleep(10)
    print(pagina_atual)
    pagina_atual+=1
ranking=ranking=pd.DataFrame.from_dict(ranking)
ranking.to_csv(path_to_save+'ranking_'+ano+'.csv',index=False)
driver.quit()
              


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [40]:
x=pd.read_csv(path_to_save+'ranking_2004.csv')
x.tail()

,World Rank,Institution,Country,Regional Rank,Total Score,Alumni,Award,HiCi,N&S,PUB,PCP
497,404-502,University of Tromso,no,4,NaN,0.0,0.0,8.7,3.0,23.3,7.8
498,404-502,University of Wuppertal,de,38-43,NaN,0.0,0.0,12.4,6.9,18.1,8.3
499,404-502,University of Zaragoza,es,5-9,NaN,0.0,0.0,8.7,2.5,27.6,8.7
500,404-502,West Virginia University,us,140-170,NaN,0.0,0.0,0.0,6.4,28.6,8.7
501,404-502,York University,ca,20-23,NaN,0.0,0.0,0.0,12.8,25.5,8.5


# 2003 a 2023

In [64]:
ano=[str(ano) for ano in np.arange(2005,2024)]
#website='https://www.shanghairanking.com/rankings/arwu/2023'
#driver = webdriver.Chrome()
#driver.get(website) #abre a pagina
#driver.maximize_window()  #maximizar a página

In [43]:
anos=[str(i) for i in range(2005,2024)]
for ano in anos:
    website='https://www.shanghairanking.com/rankings/arwu/'+ano #abrir a pagina do ano selecionado
    driver = webdriver.Chrome()
    driver.get(website) #abre a pagina
    driver.maximize_window()  #maximizar a página 
    time.sleep(20) #aguardar o carregamento da página


    #Dicionário que receberá os dados
    ranking={'World Rank' :[],
              'Institution':[],
              'Country'    :[],
              'Regional Rank':[],
              'Total Score':[],
                'Alumni': [],
                'Award' : [],
                'HiCi'  : [],
                'N&S'  : [],
                'PUB'   : [],
                'PCP'   : []}
    
    #verificar o número de páginas
    pagination=driver.find_element('xpath',"//ul[contains(@class,'pagination')]")
    paginas=pagination.find_elements('xpath','./li')
    num_paginas=paginas[-2].text
    num_paginas

    botao_next=pagination.find_element('xpath','//li[@class=" ant-pagination-next"]') #botão para avançar a página
    time.sleep(3)

    pagina_atual=1 #primera página do ranking
    while pagina_atual<=int(num_paginas): #varrer todas as páginas do ranking
        
        botoes_header=driver.find_elements(by='xpath',value='//th//img') #localizar botão dropdown para seleção das features. Ele está nos headers da tabela. 
                                                                        #Ao clicar nesse botão o menu dropdow é aberto
        botoes_header=botoes_header[-1] #nos headers estão dois botões dropdown. O primeiro seleciona o país e o segundo a feature. O botão desejado é o segundo.
                                        #botões_header é o botão qie deve ser clicado para abertura do menu dropdown
        opcoes=driver.find_elements(by='xpath',value='//th') #menus dropdown no header da tabela. opcoes são as opções contidas no menu dropdown
        opcoes=opcoes[-1] #segundo menu dropdown no header da tabela
        opcoes=opcoes.find_elements('xpath','.//li') #pegar as opções contidas no menu dropdown. Opcoes é uma lista em que cada elemento corresponde a uma feature
        
        table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem os headers
        table_rows=table.find_elements(by='xpath',value="./tr") #rows da tabela

        '''
        first_feature é usada para determinar quais colunas da tabela serão coletadas. A tabela tem várias colunas, somente em uma delas a feature na coluna pode ser selecionada.
        Quando a feature na coluna é selecionada, somente os dados da coluna são alterados. Então quando first_feature==true os dados de todas as colunas serão coletados, caso contrário
        serão coletados somentes os dados da coluna com a feature selecionada
        '''
        first_feature=True 
        for feature in features:       
          botoes_header.click() #clicar no botão para abrir o menu dropdown
          time.sleep(5)
          for opcao in opcoes: #cada opcao é uma das features no menu dropdown
            if opcao.text==feature: #opcao.text retorna o nome da feature. Seleciona a opção desejada da feature
              opcao.click()
              if first_feature: #coletar dados de todas as colunas na tabela
                for row in table_rows: #loop por todas as linhas da tabela
                  td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                  ranking['World Rank'].append(td[0].text)#primeira coluna é a posicao no ranking mundial
                  ranking['Institution'].append(td[1].text)#primeira coluna é a posicao no ranking mundial
                  codigo_pais=td[2].find_element(by='xpath',value='div[@class="region-img"]') #no pais está url de uma imagem da bandeira do pais
                  ranking['Country'].append(codigo_pais.get_attribute("style").split('/')[-1].split('.')[0]) #extrair o nome do pais daurl
                  ranking['Regional Rank'].append(td[3].text)
                  ranking['Total Score'].append(td[4].text)
                  ranking[feature].append(td[5].text)
                  first_feature=False #na próxima interação somente serão coletados os dados da feature selecionada no menu dropdown. As demais colunas da tabela já foram coletados
              else:
                for row in table_rows: #loop por todas as linhas da tabela
                  td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                  ranking[feature].append(td[5].text)  
              time.sleep(5)
              break
        time.sleep(2)
        botao_next.click() #avançar pagina
        time.sleep(2)
        botao_top=driver.find_element('xpath','//div[@class="back-top"]') #botão para voltar ao topo da página
        botao_top.click() #clicar no botão
        time.sleep(5)     
        print('Página atual: {}'.format(pagina_atual))
        print('Ano: {}'.format(ano))
        print('')
        pagina_atual+=1
    ranking=pd.DataFrame.from_dict(ranking)
    ranking.to_csv(path_to_save+'ranking_'+ano+'.csv',index=False)
    driver.quit()  
        
        
    



 

Página atual: 1
Ano: 2005

Página atual: 2
Ano: 2005



KeyboardInterrupt: 